In [ ]:
!pip install --upgrade openai

We are using list of sentences as the input here

In [ ]:
from openai import OpenAI
from google.colab import userdata

client = OpenAI(api_key=userdata.get('gpt-key'))

def llm_score_for_list(text_list, model, temp):
  completion = client.chat.completions.create(
      model=model,
      temperature = temp,
      n = 1,
      messages=[
        {"role": "system", "content": """You are a financial expert. Your task is to classify the financial sentiment of a given input text.
                                        For a given input, you need to output a single number only which is a sentiment score between -1 and 1.
                                        -1 corresponds to completely negative sentiment, +1 corresponds to completely positive sentiment."""},
        {"role": "user",  "content": f"""Classify the financial sentiment of the following sentences included in a list.
                                      Give a score between -1 and 1 for each element of the list. Output only the scores as a list!
                                      {text_list}"""}
      ]
    )

  return completion

In [ ]:
# importing the labeled sentences from the 10K reports as a dictionary
import json
with open('./combined_labeled_sentences.json', 'r', encoding='utf-8') as f:
    sentences_dict = json.load(f)

In [ ]:
reports = list(sentences_dict.keys())

print(reports)

['abt22_sentences.json', 'abt23_sentences.json', 'dal22_sentences.json', 'dal23_sentences.json', 'ehc22_sentences.json', 'ehc23_sentences.json', 'f22_sentences.json', 'f23_sentences.json', 'fdx22_sentences.json', 'fdx23_sentences.json', 'holx22_sentences.json', 'holx23_sentences.json', 'ogn22_sentences.json', 'ogn23_sentences.json', 'tsla22_sentences.json', 'tsla23_sentences.json', 'ups22_sentences.json', 'ups23_sentences.json', 'zbh22_sentences.json', 'zbh23_sentences.json']


In [ ]:
len(sentences_dict["tsla22_sentences.json"])

2078

In [ ]:
# selecting the relevant-labeled sentences
for c in list(sentences_dict.keys()):
  sentences_dict.update({c: [s[0] for s in sentences_dict[c] if s[1] == 1]})

In [ ]:
len(sentences_dict["tsla22_sentences.json"])

439

In [ ]:
print(sentences_dict["tsla22_sentences.json"])

['The aggregate market value of voting stock held by non-affiliates of the registrant, as of June 30, 2022, the last day of the registrant’s most recently completed second fiscal quarter, was', '$580.48 billion (based on the closing price for shares of the registrant’s Common Stock as reported by the NASDAQ Global Select Market on June 30, 2022).', 'These forward-looking statements include, but are not limited to, statements concerning any potential future impact of the coronavirus disease (“COVID-19”) pandemic on our business, supply chain constraints, our strategy, competition, future operations and production capacity, future financial position, future revenues, projected costs, profitability, expected cost reductions, capital adequacy, expectations regarding demand and acceptance for our technologies, growth opportunities and trends in the markets in which we operate, prospects and plans and objectives of management.', 'We also strive to lower the cost of ownership for our customer

In [ ]:
# calculation of avg. sentiment scores
# takes ~30min, 4 USD
import numpy as np

T = 0
model = "gpt-4o"
chunk_size = 10

score_dict = {}
for c in list(sentences_dict.keys()):
  text_list = [sentences_dict[c][i:i + chunk_size] for i in range(0, len(sentences_dict[c]), chunk_size)]
  score_list = []
  for l in text_list:
    scores = np.array(eval( llm_score_for_list(l, model, T).choices[0].message.content ))
    score_list = np.append(score_list, scores)
  avg_score = np.mean(score_list)
  score_dict[c] = avg_score

In [ ]:
print(score_dict)

{'abt22_sentences.json': -0.13821138211382114, 'abt23_sentences.json': -0.23224932249322497, 'dal22_sentences.json': -0.19101654846335697, 'dal23_sentences.json': -0.188, 'ehc22_sentences.json': -0.35683229813664596, 'ehc23_sentences.json': -0.37304860088365244, 'f22_sentences.json': -0.24964028776978417, 'f23_sentences.json': -0.3013392857142857, 'fdx22_sentences.json': -0.1740425531914893, 'fdx23_sentences.json': -0.29590643274853795, 'holx22_sentences.json': -0.3357815442561205, 'holx23_sentences.json': -0.3319470699432892, 'ogn22_sentences.json': -0.381021897810219, 'ogn23_sentences.json': -0.42777777777777776, 'tsla22_sentences.json': -0.2136674259681093, 'tsla23_sentences.json': -0.2537190082644628, 'ups22_sentences.json': -0.17121212121212123, 'ups23_sentences.json': -0.26546112115732373, 'zbh22_sentences.json': -0.39709443099273606, 'zbh23_sentences.json': -0.33834196891191715}


In [ ]:
import numpy as np

T = 0
model = "gpt-3.5-turbo"
chunk_size = 10

score_dict = {}
for c in list(sentences_dict.keys()):
  text_list = [sentences_dict[c][i:i + chunk_size] for i in range(0, len(sentences_dict[c]), chunk_size)]
  score_list = []
  for l in text_list:
    scores = np.array(eval( llm_score_for_list(l, model, T).choices[0].message.content ))
    score_list = np.append(score_list, scores)
  avg_score = np.mean(score_list)
  score_dict[c] = avg_score

In [ ]:
print(score_dict)

{'abt22_sentences.json': 0.153921568627451, 'abt23_sentences.json': 0.059831460674157316, 'dal22_sentences.json': 0.06824817518248176, 'dal23_sentences.json': 0.09811046511627906, 'ehc22_sentences.json': -0.05732283464566929, 'ehc23_sentences.json': -0.07998500749625187, 'f22_sentences.json': 0.012259970457902529, 'f23_sentences.json': -0.03361669242658424, 'fdx22_sentences.json': 0.09758771929824561, 'fdx23_sentences.json': -0.05369261477045907, 'holx22_sentences.json': -0.04148936170212765, 'holx23_sentences.json': -0.051277013752455795, 'ogn22_sentences.json': -0.0788718929254302, 'ogn23_sentences.json': -0.07284090909090908, 'tsla22_sentences.json': 0.04707259953161593, 'tsla23_sentences.json': 0.041666666666666685, 'ups22_sentences.json': 0.11345029239766083, 'ups23_sentences.json': 0.0410351201478743, 'zbh22_sentences.json': -0.08345864661654134, 'zbh23_sentences.json': -0.022593582887700542}


average only the top 10% + bottom 10%?





In [ ]:
import math
import numpy as np

T = 0
model = "gpt-4o"
chunk_size = 10

score_dict_2 = {}
for c in list(sentences_dict.keys()):
  text_list = [sentences_dict[c][i:i + chunk_size] for i in range(0, len(sentences_dict[c]), chunk_size)]
  score_list = []
  for l in text_list:
    scores = np.array(eval( llm_score_for_list(l, model, T).choices[0].message.content ))
    score_list = np.append(score_list, scores)
  j = math.ceil(len(score_list)/10)
  avg_score = np.mean(np.concatenate((np.sort(score_list)[:j], np.sort(score_list)[-j:])))  # avg. of top 10% and bottom 10% only
  score_dict_2[c] = avg_score

In [ ]:
print(score_dict_2)

{'abt22_sentences.json': -0.08378378378378373, 'abt23_sentences.json': -0.09054054054054053, 'dal22_sentences.json': -0.036046511627907014, 'dal23_sentences.json': -0.06428571428571433, 'ehc22_sentences.json': -0.13230769230769232, 'ehc23_sentences.json': -0.11323529411764705, 'f22_sentences.json': -0.14285714285714302, 'f23_sentences.json': -0.14999999999999994, 'fdx22_sentences.json': -0.04787234042553188, 'fdx23_sentences.json': -0.09423076923076922, 'holx22_sentences.json': -0.15000000000000005, 'holx23_sentences.json': -0.1113207547169812, 'ogn22_sentences.json': -0.20545454545454547, 'ogn23_sentences.json': -0.19666666666666666, 'tsla22_sentences.json': -0.09431818181818183, 'tsla23_sentences.json': -0.10405405405405399, 'ups22_sentences.json': -0.10660377358490566, 'ups23_sentences.json': -0.15982142857142861, 'zbh22_sentences.json': -0.1988095238095238, 'zbh23_sentences.json': -0.15256410256410258}
